# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction: Business Problem <a name="introduction"></a>

The objective of this project is to develop a tool that will help 2 immigrants identify the best location for opening a new ethinc resturant in New York City. The first immigrant is Mr. Lucino, a Polish immigrant, who wants to open a new Polish resturant and Mrs. Jundi, a Lebanese immigrant, who wants to open a new lebanese resturant. The new immigrants are overwhelmed looking for the best location in the NY, they want this tool to help them narrow down their options by determining the best neighborhood to open their resturants.   

## Data <a name="data"></a>

The tool will use NY city neighborhoods and boroughs database used in Week 3 lab for the list of neighborhoods and their coordinates. The database was exported as CSV file and used here. Foursquare API will be used to get the venues information. The tool will correlate all the venues categories to identify venues that open in the same proximity then it will identify neighborhoods where the new resturant category (Lebanese or Polish) is lacking.

The factors that will be considered are:
* Availablility of other venues that open within proximity of a Lebanese and Polish resturants.
* Number of existing Lebanese or Polish resturants in the neighborhood.

First, lets download all the dependencies we need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


Then we upload NY neighboorhoods data from CSV file.

In [2]:
hoods = pd.read_csv('NY Neighborhoods.csv')
hoods.head(10)

,Unnamed: 0,Borough,Neighborhood,Latitude,Longitude
0,0,Bronx,Wakefield,40.894705,-73.847201
1,1,Bronx,Co-op City,40.874294,-73.829939
2,2,Bronx,Eastchester,40.887556,-73.827806
3,3,Bronx,Fieldston,40.895437,-73.905643
4,4,Bronx,Riverdale,40.890834,-73.912585
5,5,Bronx,Kingsbridge,40.881687,-73.902818
6,6,Manhattan,Marble Hill,40.876551,-73.910660
7,7,Bronx,Woodlawn,40.898273,-73.867315
8,8,Bronx,Norwood,40.877224,-73.879391
9,9,Bronx,Williamsbridge,40.881039,-73.857446


Here we define our Foursquare API credentials.

In [3]:
CLIENT_ID = 'W5Y2SJWEORJOICYFVOD4PQNIWW3DO0NKLT302I21GMAM3CQB' # your Foursquare ID
CLIENT_SECRET = 'K3ILU01A43TQRFNXAABVNDFLKK5INNLFXXFII2GN0PK15PNF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

We create **getNearbyVenues** function that we will use to retreive venues information for each neighborhood.

In [4]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

We use **getNearbyVenues** to get the list of top 100 venues in each neighborhood in NY. 

In [5]:
venues = getNearbyVenues(names=hoods['Neighborhood'],
                                   latitudes=hoods['Latitude'],
                                   longitudes=hoods['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

let's examine the shape and data of **venues** dataframe

In [6]:
print(venues.shape)
venues.head(5)

(10177, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
3,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


Let's see how many unique venue categories there are

In [7]:
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 439 uniques categories.


## Methodology <a name="methodology"></a>

We will use the venue categories to create 2 venue profiles of each neighborhood. One for the count of the venues and one for the mean. The mean profile will be used to corrolate then venues categories to identify venues that open in the same proximity of Lebanese and Polish restaurants. The count profile will be used to identify the neighborhoods where Lebanese and Polish restaurants are lacking. The neigborhoods will be ranked depending on their profile corrolation and availability of Polish or Lebanese restaurants. Two maps will be generated with markers that indicate the neighborhoods rankings. 


## Analysis <a name="analysis"></a>

Now, we will create a new dataframe with the neighborhood and the number of venues in each category  

In [8]:
hotVenues = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
hotVenues['Neighborhood'] = venues['Neighborhood'] 

# move neighborhood column to the first column
#fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
#toronto_onehot = toronto_onehot[fixed_columns]
mid = hotVenues['Neighborhood']
hotVenues.drop(labels=['Neighborhood'], axis=1,inplace = True)
hotVenues.insert(0, 'Neighborhood', mid)
hotVenues.head()

,Neighborhood,ATM,Accessories Store,Acupuncturist,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Marine Terminal,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle House,North Ind

Then we create two grouped datasets, one grouped by the mean of the venues count (meanHoods) and one by the count of venues (countHoods). meanHoods will be used to correlate venues and count to determine neighborhoods with low Polish or Lebanese resturants. 

In [9]:
meanHoods = hotVenues.groupby('Neighborhood').mean().reset_index()
print(meanHoods.shape)
meanHoods.head()


(302, 439)


,Neighborhood,ATM,Accessories Store,Acupuncturist,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Marine Terminal,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle House,North Ind

In [10]:
countHoods = hotVenues.groupby('Neighborhood').sum().reset_index()
print(countHoods.shape)
countHoods.head()

(302, 439)


,Neighborhood,ATM,Accessories Store,Acupuncturist,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Marine Terminal,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle House,North Ind

Then we create normalized dataframes for the correlation of venues with Lebanese and Polish resturants.

In [207]:
from sklearn import preprocessing

venuesCorr = meanHoods.corr()
lebCorr = venuesCorr[["Lebanese Restaurant"]]
polCorr = venuesCorr[[ "Polish Restaurant"]]


# Normalizing the dataframes

lebArr = lebCorr.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
lebArr = min_max_scaler.fit_transform(lebArr)
lebCorr["Weight"] = lebArr
lebCorr = lebCorr.drop(columns = "Lebanese Restaurant")
lebCorr = lebCorr.drop("Lebanese Restaurant") #remove Lebanese Restaurant form the list




polArr = polCorr.values #returns a numpy array

polArr = min_max_scaler.fit_transform(polArr)
polCorr["Weight"] = polArr
polCorr = polCorr.drop(columns = "Polish Restaurant")
polCorr = polCorr.drop("Polish Restaurant")#remove Polish Restaurant form the list

print(lebCorr)
print(polCorr)

                                            Weight
ATM                                       0.044142
Accessories Store                         0.128230
Acupuncturist                             0.044142
Adult Boutique                            0.322458
Afghan Restaurant                         0.042205
African Restaurant                        0.043398
American Restaurant                       0.054035
Antique Shop                              0.040049
Arcade                                    0.043319
Arepa Restaurant                          0.038612
Argentinian Restaurant                    0.035121
Art Gallery                               0.028397
Art Museum                                0.041968
Arts & Crafts Store                       0.038151
Arts & Entertainment                      0.042302
Asian Restaurant                          0.049422
Athletics & Sports                        0.034779
Auditorium                                0.044142
Australian Restaurant          

<ipython-input-207-95fb74ce3c21>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lebCorr["Weight"] = lebArr
<ipython-input-207-95fb74ce3c21>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  polCorr["Weight"] = polArr


Lets explore the top 10 venues for each type of restaurants.

In [208]:
topleb = lebCorr.nlargest(10,"Weight")


toppol = polCorr.nlargest(10,"Weight")


print(topleb)
print(toppol)

                       Weight
Persian Restaurant   0.720229
Shoe Repair          0.720229
English Restaurant   0.720229
Leather Goods Store  0.521138
Strip Club           0.521138
Udon Restaurant      0.521138
Beer Bar             0.358172
Adult Boutique       0.322458
Gaming Cafe          0.277542
Bike Trail           0.270431
                             Weight
Noodle House               0.510459
Outdoors & Recreation      0.387289
New American Restaurant    0.351403
Coworking Space            0.296614
Mediterranean Restaurant   0.269534
Middle Eastern Restaurant  0.258932
Taco Place                 0.246654
German Restaurant          0.217457
Garden Center              0.204202
Vape Store                 0.189293


From this we can confirm that Lebanese Resturants are usually in neighborhoods that has Persian and English Restaurants as well as Shoe repair and Leather Goods shops. While Polish Restaurants are in neighborhoods with Noodle Houses, Outdoors and Recreation, New American restaurants and Coworking spaces.  

Now we will correlate the retaurants profiles with venues means in each neighborhood to see how similar the neighboorhood venues are to the venues that open near Lebanese and Polish restaurants.    

In [209]:
lebHoods = meanHoods.drop(["Lebanese Restaurant"],1)#remove Lebanese Restaurant since the value will be 1
lebHoods = lebHoods.T # transpose the dataframe to match the shape of the restaurant profile matrix
lebHoods.columns = lebHoods.iloc[0]
lebHoods = lebHoods.drop("Neighborhood", axis = 0)# make Neighborhoods as teh column headers

lebHoods = pd.DataFrame(lebHoods)# convert to pandas DataFrame

lebHoodsCorr = pd.DataFrame(index =lebHoods.columns, columns = ["Value"] ) # create a dataframe for the neighborhoods correlations with Lebanese restaurants profiles

for i in range(len(lebHoods.columns)):
     lebHoodsCorr.loc[lebHoods.columns[i]] = lebHoods[lebHoods.columns[i]].astype('float64').corr(lebCorr["Weight"].astype('float64')) #correlate the neighborhoods with the profiles

        
polHoods = meanHoods.drop(["Polish Restaurant"],1)#remove Polish Restaurant since the value will be 1
polHoods = polHoods.T # transpose the dataframe to match the shape of the restaurant profile matrix
polHoods.columns = polHoods.iloc[0]
polHoods = polHoods.drop("Neighborhood", axis = 0)# make Neighborhoods as teh column headers

polHoods = pd.DataFrame(polHoods)# convert to pandas DataFrame

polHoodsCorr = pd.DataFrame(index =polHoods.columns, columns = ["Value"] ) # create a dataframe for the neighborhoods correlations with Polish restaurants profiles

for i in range(len(polHoods.columns)):
     polHoodsCorr.loc[polHoods.columns[i]] = polHoods[polHoods.columns[i]].astype('float64').corr(polCorr["Weight"].astype('float64')) #correlate the neighborhoods with the profiles



let's explore the top 10 neighborhoods for Lebanese restaurants

In [210]:
lebHoodsCorr.sort_values("Value", ascending=False).head(10)

,Value
Neighborhood,
Sutton Place,0.293417
Greenwich Village,0.228352
Soho,0.103279
Noho,0.082809
West Village,0.062787
Upper West Side,0.050469
Astoria,0.050225
Civic Center,0.048836
Egbertville,0.044713


let's explore the top 10 neighborhoods for Polish restaurants

In [211]:
polHoodsCorr.sort_values("Value", ascending=False).head(10)


,Value
Neighborhood,
Forest Hills Gardens,0.388636
Arrochar,0.293602
Ridgewood,0.221355
Astoria,0.164868
West Village,0.150217
Greenpoint,0.137565
East Harlem,0.132904
Kensington,0.132519
Noho,0.126034


let's normalize the two correlation dataframes 

In [213]:
lebArr = lebHoodsCorr.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
lebArr = min_max_scaler.fit_transform(lebArr)
lebHoodsCorr["Value"] = lebArr


polArr = polHoodsCorr.values #returns a numpy array
polArr = min_max_scaler.fit_transform(polArr)
polHoodsCorr["Value"] = polArr


At this point, we have identified the neighborhoods that are correlated with the restaurants. Now we need to identify the availability of Lebanese and Polish Restaurants in the neighborhoods.

we will do this by creating two normalized availability dataframes for the count of lebanese and polish restaurants. Higher availability values for a neighborhood indicates higher number of lebanse or polish restaurants.

In [214]:

lebCount = countHoods[["Neighborhood","Lebanese Restaurant"]].copy()
lebCount = lebCount.set_index("Neighborhood")
lebArr = lebCount.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
lebArr = min_max_scaler.fit_transform(lebArr)
lebCount["Avail"] = lebArr
lebCount = lebCount.drop(["Lebanese Restaurant"],axis = 1)


polCount = countHoods[["Neighborhood","Polish Restaurant"]].copy()
polCount = polCount.set_index("Neighborhood")
polArr = polCount.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
polArr = min_max_scaler.fit_transform(polArr)
polCount["Avail"] = polArr
polCount = polCount.drop(["Polish Restaurant"],axis = 1)


Finally, we will calculate a score for each neighborhood based on their venues profile and availability of the restaurants types. The score is calculated by multiplyting the value of correlation by (1-availability). Neighborhoods with higher number of restaurant type will score lower.

In [235]:
lebHoodsRank = pd.DataFrame(columns = ["Score"], index = lebCount.index) # create a dataframe for the neighborhoods Scores for Lebanese Restaurants
for i in range(len(lebCount.index)):
    lebHoodsRank.loc[lebCount.index[i]] = lebHoodsCorr.loc[lebCount.index[i]]["Value"] * (1-lebCount.loc[lebCount.index[i]]["Avail"])
    


polHoodsRank = pd.DataFrame(columns = ["Score"], index = polCount.index) # create a dataframe for the neighborhoods Scores for Polish Restaurants
for i in range(len(polCount.index)):
    polHoodsRank.loc[polCount.index[i]] = polHoodsCorr.loc[polCount.index[i]]["Value"] * (1-polCount.loc[polCount.index[i]]["Avail"])
    

lets explore the top 10 neighborhoods for Lebanese restaurants and compare it with the correlations rank to see the effect of availability.

In [242]:
lebHoodsExp = pd.DataFrame(columns = ["Score","Corr","Avail"], index = lebCount.index)
lebHoodsExp["Score"] = lebHoodsRank["Score"]
lebHoodsExp["Corr"] = lebHoodsCorr["Value"]
lebHoodsExp["Avail"] = lebCount["Avail"]

lebHoodsExp.sort_values("Avail", ascending = False).head()

,Score,Corr,Avail
Neighborhood,,,
Greenwich Village,0.0,0.829073,1.0
Sutton Place,0.0,1.000000,1.0
Allerton,0.035489,0.035489,0.0
Ozone Park,0.040313,0.040313,0.0
Olinville,0.10001,0.100010,0.0


We can see that accounting for the availability has exclueded Sutton Place and Greenwich Village even though they are highly correlated with Lebanese Restaurants profile.

In [244]:
polHoodsExp = pd.DataFrame(columns = ["Score","Corr","Avail"], index = polCount.index)
polHoodsExp["Score"] = polHoodsRank["Score"]
polHoodsExp["Corr"] = polHoodsCorr["Value"]
polHoodsExp["Avail"] = polCount["Avail"]

polHoodsExp.sort_values("Avail", ascending = False).head(10)


,Score,Corr,Avail
Neighborhood,,,
Greenpoint,0.0,0.479657,1.0
Arrochar,0.401521,0.803042,0.5
Downtown,0.178483,0.356967,0.5
Ridgewood,0.326656,0.653312,0.5
Forest Hills Gardens,0.5,1.000000,0.5
Allerton,0.074009,0.074009,0.0
Ocean Hill,0.289871,0.289871,0.0
Paerdegat Basin,0.112729,0.112729,0.0
Ozone Park,0.100603,0.100603,0.0


For the Polish restaurant, Greenpoint has been excluded while Arrochar, Downtown, Ridgewood and Forest Hills Gardens scores have been lowered since they aleady have Polish restaurants.  

Lets explore the top 10 neighborhoods scores for Lebanese restaurants.

In [251]:
lebHoodsExp.sort_values("Score",ascending = False).head(10)

,Score,Corr,Avail
Neighborhood,,,
Soho,0.500507,0.500507,0.0
Noho,0.446732,0.446732,0.0
West Village,0.394136,0.394136,0.0
Upper West Side,0.361777,0.361777,0.0
Astoria,0.361135,0.361135,0.0
Civic Center,0.357485,0.357485,0.0
Egbertville,0.346655,0.346655,0.0
Carroll Gardens,0.340191,0.340191,0.0
Flatiron,0.338216,0.338216,0.0


We can see that Soho, Noho and West Village are teh best location to open a new Lebanese Restaurant because they are highly correlated with the profile of a Lebanese restaurant and currently does not have any lebanese restaurants.   

Now, lets explore the top 10 neighborhoods scores for Lebanese restaurants.

In [250]:
polHoodsExp.sort_values("Score",ascending = False).head(10)

,Score,Corr,Avail
Neighborhood,,,
Astoria,0.536243,0.536243,0.0
West Village,0.505879,0.505879,0.0
Forest Hills Gardens,0.5,1.000000,0.5
East Harlem,0.469997,0.469997,0.0
Kensington,0.469199,0.469199,0.0
Noho,0.455759,0.455759,0.0
Little Italy,0.453306,0.453306,0.0
Auburndale,0.448815,0.448815,0.0
Bushwick,0.423651,0.423651,0.0


This tell us that the best neighborhoods for a new Polish restaurant are Astoria, West Village, and Forest Hills Gardens because they are highly correlated with the Polish restaurant profile and with the exception of Forest Hills Gardens have no Polish restaurants. 

Now Lets create a map to visually represent the neighborhoods score. First of all, we will create a dataframe with the scores and the Neighborhoods longitudes and latitudes. 

In [311]:
mapHoods = pd.DataFrame(columns = ["Neighborhood","Latitude","Longitude","Leb Score","Leb Color","Pol Score", "Pol Color"])

mapHoods[["Neighborhood","Latitude","Longitude"]] = hoods[["Neighborhood","Latitude","Longitude"]]
mapHoods = mapHoods.set_index("Neighborhood")
mapHoods["Leb Score"] = lebHoodsExp["Score"]
mapHoods["Pol Score"] = polHoodsExp["Score"]
mapHoods = mapHoods.reset_index()

mapHoods["Leb Color"] = pd.cut(mapHoods["Leb Score"], bins=4, labels=['red',  'orange','yellow' ,'green'])
mapHoods["Pol Color"] = pd.cut(mapHoods["Pol Score"], bins=4, labels=['red',  'orange','yellow' ,'green'])

mapHoods.head()

,Neighborhood,Latitude,Longitude,Leb Score,Leb Color,Pol Score,Pol Color
0,Wakefield,40.894705,-73.847201,0.108154,red,0.218721,orange
1,Co-op City,40.874294,-73.829939,0.073231,red,0.187542,orange
2,Eastchester,40.887556,-73.827806,0.056022,red,0.099989,red
3,Fieldston,40.895437,-73.905643,0.141728,orange,0.220873,orange
4,Riverdale,40.890834,-73.912585,0.121452,red,0.19621,orange


Then we will create the map for Lebanese Restaurants 

In [312]:

map_newyork = folium.Map(location=[40.710318150000006, -74.00496837048613], zoom_start=10)

for lat, lng, neighborhood, score, clr in zip(mapHoods['Latitude'], mapHoods['Longitude'], mapHoods['Neighborhood'],mapHoods['Leb Score'],mapHoods['Leb Color'] ):
    label = '{},Score: {}'.format(neighborhood, score)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color= clr,
        fill=True,
        fill_color=clr,
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

We will create a map for Polish Restaurants.

In [313]:
map_newyork = folium.Map(location=[40.710318150000006, -74.00496837048613], zoom_start=10)

for lat, lng, neighborhood, score, clr in zip(mapHoods['Latitude'], mapHoods['Longitude'], mapHoods['Neighborhood'],mapHoods['Pol Score'],mapHoods['Pol Color'] ):
    label = '{},Score: {}'.format(neighborhood, score)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color= clr,
        fill=True,
        fill_color=clr,
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

## Results and Discussion <a name="results"></a>

### Lebanese Restaurant

By examining the neighborhoods rankings we can see that Soho, Noho and West Village are the best neighbohoods to open a new Lebanese Restaurant because they are highly correlated with the profile of a Lebanese restaurant and currently does not have any lebanese restaurants. In addition, by examining the map, we can see that Manhattan is the best Borough to open a Lebanese Restaurant. 

### Polish Restaurant

Neighborhoods rankings tell us that the best neighborhoods for a new Polish restaurant are Astoria, West Village, and Forest Hills Gardens because they are highly correlated with the Polish restaurant profile and with the exception of Forest Hills Gardens have no Polish restaurants. By examining the map, we can see that for Polish Restaurants the best borough is also Manhattan with few neighborhoods in other boroughs such as Bushwick in Brooklyn, Bayside in Queens and Fox Hills in Staten Island.

## Conclusion <a name="conclusion"></a>

 The objective of this project is to develop a tool that will help a Lebanese and a Polish immigrants immigrants identify the best neighborhood for opening a new ethnic resturant in New York City. The project utilized New York neighborhood information used in Week 3 as well as Foursquare API to collect neighborhoods venues information. Profiles for Lebanese and Polish restuarnts were creatare Astoria, West Village, and Forest Hills Gardensed to correlate the neighborhoods venues with the profiles. Then a score for each neighborhoods was calculated based on its correlation with the restaurant profile and availability of the type of resturants. finally, maps were created to visualize the neighborhoods and their scores to get a birds eye view of the results.
 
The project concluded that Manhattan is the best Borough for both types of restaurants. For Lebanese Restaurants, the best neighborhoods are Soho, Noho and West Village (neighborhoods in west Manhattan). For Polish Restaurants,the best neighborhoods are Astoria, West Village, and Forest Hills Gardens.
 